In [25]:
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fairness_checker import *

In [26]:
# To load the model back
with open('./model/compas_RF.pkl', 'rb') as f:
    clf = pickle.load(f)
with open('./model/compas_synth.pkl', 'rb') as f:
    model = pickle.load(f)

In [27]:
class model_wrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, file):
        df = pd.read_csv(file)
        X = self.preprocessing(df)

        return self.model.predict(X)

    def preprocessing(self, df):
        df = df[['age', 'sex', 'decile_score', 'priors_count', 'race', 'days_in_jail', 'c_days_from_compas', 'is_violent_recid', 'v_decile_score']].copy()

        return df

trained = model_wrapper(clf)

In [28]:
iter = 100

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for _ in range(iter):
    synth = model.synthetic_data(rows=7000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['is_recid'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

In [29]:
og_disparate_impact = 0
og_demographic_parity = 0
og_equalized_odds_1, og_equalized_odds_2 = 0, 0
og_equal_opportunity = 0
og_accuracy_eqaulity = 0
og_predictive_parity = 0
og_equal_calibration = 0
og_conditional_statistical_parity = 0
og_predictive_equality = 0
og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2 = 0, 0
og_positive_balance = 0
og_negative_balance = 0
og_mean_difference = 0

cog = fairness_model_checker("./data/compas_processed.csv", verbose=False)

privileged_predicate = lambda row: row['sex'] != '0'
positive_predicate = lambda Y: Y == 1
truth_predicate = lambda row: row['is_recid'] == '1'

og_disparate_impact = cog.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True)
og_demographic_parity = cog.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True)
(og_equalized_odds_1, og_equalized_odds_2) = cog.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_equal_opportunity = cog.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_accuracy_eqaulity = cog.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_predictive_parity = cog.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# equal_calibration (n/a)
og_conditional_statistical_parity = cog.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True)
og_predictive_equality = cog.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
(og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2) = cog.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# positive_balance (n/a)
# negative_balance (n/a)
og_mean_difference = cog.mean_difference(0.2, trained, privileged_predicate, positive_predicate, value=True)

In [30]:
print("name               ", "og          ", "synth       ", "diff        ", "std         ", "ratio")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", f"{np.std(np.array(demographic_parity)):.10f}", f"{og_demographic_parity / np.mean(np.array(demographic_parity)):.10f}")
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", f"{np.std(np.array(accuracy_eqaulity)):.10f}", f"{og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity)):.10f}")
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", f"{np.std(np.array(equalized_odds_1)):.10f}", f"{og_equalized_odds_1 / np.mean(np.array(equalized_odds_1)):.10f}")
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", f"{np.std(np.array(equalized_odds_2)):.10f}", f"{og_equalized_odds_2 / np.mean(np.array(equalized_odds_2)):.10f}")
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}")
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}")
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", f"{np.std(np.array(mean_difference)):.10f}", f"{og_mean_difference / np.mean(np.array(mean_difference)):.10f}")

print(
"sum of diff:",
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

print(
"avg of ratio:",
(og_demographic_parity / np.mean(np.array(demographic_parity))+
og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity))+
og_equalized_odds_1 / np.mean(np.array(equalized_odds_1))+
og_equalized_odds_2 / np.mean(np.array(equalized_odds_2))+
og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1))+
og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2))+
og_mean_difference / np.mean(np.array(mean_difference))) / 7
)

name                og           synth        diff         std          ratio
demographic_parity  0.1310570603 0.0980059055 0.0330511548 0.0141631098 1.3372363602
accuracy_eqaulity   0.0079385825 0.0129970853 0.0050585028 0.0102108821 0.6107971357
equalized_odds_1    0.0249050732 0.0936224133 0.0687173402 0.0189836669 0.2660161415
equalized_odds_2    0.0177674779 0.1016472155 0.0838797376 0.0202764525 0.1747955203
accuracy_equality_1 0.1709137581 0.0505556965 0.1203580616 0.0411999381 3.3807022763
accuracy_equality_2 0.1695760037 0.0303041474 0.1392718564 0.0255176411 5.5958018433
mean_difference     0.3282008342 0.2274669242 0.1007339100 0.0081396368 1.4428508028
sum of diff: 0.5510705634113173
avg of ratio: 1.8297428685653352


# Without heuristic

In [31]:
with open('./model/compas_synth_mst.pkl', 'rb') as f:
    model = pickle.load(f)

In [33]:
iter = 100

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for _ in range(iter):
    synth = model.synthetic_data(rows=7000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['is_recid'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    # conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

In [ ]:
print("name               ", "og          ", "synth       ", "diff        ", "std         ", "ratio")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", f"{np.std(np.array(demographic_parity)):.10f}", f"{og_demographic_parity / np.mean(np.array(demographic_parity)):.10f}")
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", f"{np.std(np.array(accuracy_eqaulity)):.10f}", f"{og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity)):.10f}")
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", f"{np.std(np.array(equalized_odds_1)):.10f}", f"{og_equalized_odds_1 / np.mean(np.array(equalized_odds_1)):.10f}")
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", f"{np.std(np.array(equalized_odds_2)):.10f}", f"{og_equalized_odds_2 / np.mean(np.array(equalized_odds_2)):.10f}")
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}")
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}")
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", f"{np.std(np.array(mean_difference)):.10f}", f"{og_mean_difference / np.mean(np.array(mean_difference)):.10f}")

print(
"sum of diff:",
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

print(
"avg of ratio:",
(og_demographic_parity / np.mean(np.array(demographic_parity))+
og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity))+
og_equalized_odds_1 / np.mean(np.array(equalized_odds_1))+
og_equalized_odds_2 / np.mean(np.array(equalized_odds_2))+
og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1))+
og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2))+
og_mean_difference / np.mean(np.array(mean_difference))) / 7
)

name                og           synth        diff         std          ratio
demographic_parity  0.1310570603 0.0809558437 0.0501012166 0.0130148245 1.6188709090
accuracy_eqaulity   0.0079385825 0.0146621997 0.0067236172 0.0098610494 0.5414318887
equalized_odds_1    0.0249050732 0.0802349014 0.0553298282 0.0101318531 0.3104019913
equalized_odds_2    0.0177674779 0.0825768125 0.0648093346 0.0231703961 0.2151630389
accuracy_equality_1 0.1709137581 0.0551945997 0.1157191584 0.0364861989 3.0965666745
accuracy_equality_2 0.1695760037 0.0256428640 0.1439331397 0.0136032071 6.6129900097
mean_difference     0.3282008342 0.2220158446 0.1061849895 0.0087370522 1.4782766281
sum of diff: 0.5428012841829764
avg of ratio: 1.981957305732594
